# Metocean Data

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')
import xarray as xr
import distributed

In [ ]:
client = distributed.Client()
client

# CANADIAN DATA

## GIOPS

http://dd.weather.gc.ca/model_giops/doc/README_GIOPS.txt

## RIOPS

In [ ]:
ds = xr.open_dataset('http://navigator.oceansdata.ca/thredds/dodsC/riops/riopsf/2016091918_006_3D_nps.nc',
                       chunks={'depth':1, 'yc':200, 'xc':200})

In [ ]:
ncfiles = ['http://navigator.oceansdata.ca/thredds/dodsC/riops/riopsf/2016091906_006_3D_nps.nc',
           'http://navigator.oceansdata.ca/thredds/dodsC/riops/riopsf/2016091912_006_3D_nps.nc',
           'http://navigator.oceansdata.ca/thredds/dodsC/riops/riopsf/2016091918_006_3D_nps.nc']

In [ ]:
ds = xr.open_mfdataset(ncfiles, chunks={'depth':10, 'yc':400, 'xc':400})

In [ ]:
ds.vomecrty

In [ ]:
vx = ds.vozocrtx.isel(depth=0).sel(xc=slice(6e6-1, 8e6-1),
                              yc=slice(2e6-1, 4e6-1))
vy = ds.vomecrty.isel(depth=0).sel(xc=slice(6e6-1, 8e6-1),
                              yc=slice(2e6-1, 4e6-1))

In [ ]:
v = np.sqrt(vx**2+vy**2)

In [ ]:
v

In [ ]:
v.isel(time=2).plot(vmax=0.8)

In [ ]:
ds.latitude.sel(xc=slice(6e6, 8e6), yc=slice(2e6, 4e6))

# AMERICAN DATA

## NOMADS - GFS

- 3 hour

In [2]:
gfs_url ='https://www.ncei.noaa.gov/thredds/dodsC/gfs-g4-anl-files/201512/20151231/gfsanl_4_20151231_1800_006.grb2'

In [3]:
gfs_ds = xr.open_dataset(gfs_url)

In [4]:
gfs_ds

<xarray.Dataset>
Dimensions:                                                                   (altitude_above_msl: 3, depth_below_surface_layer: 4, depth_below_surface_layer_bounds_1: 2, height_above_ground: 1, height_above_ground1: 2, height_above_ground2: 1, height_above_ground3: 3, height_above_ground4: 3, height_above_ground_layer: 1, height_above_ground_layer1: 1, height_above_ground_layer1_bounds_1: 2, height_above_ground_layer_bounds_1: 2, isobaric: 12, isobaric1: 1, isobaric2: 21, isobaric3: 26, lat: 361, lon: 720, potential_vorticity_surface: 2, pressure_difference_layer: 1, pressure_difference_layer1: 1, pressure_difference_layer1_bounds_1: 2, pressure_difference_layer2: 2, pressure_difference_layer2_bounds_1: 2, pressure_difference_layer_bounds_1: 2, sigma: 1, sigma_layer: 4, sigma_layer_bounds_1: 2, time: 1, time1: 1, time_bounds_1: 2)
Coordinates:
  * lat                                                                       (lat) float32 90.0 ...
  * lon                  

In [5]:
u_wind = gfs_ds.__getitem__('u-component_of_wind_height_above_ground')
v_wind = gfs_ds.__getitem__('v-component_of_wind_height_above_ground')

In [6]:
u_wind.dims

('time1', 'height_above_ground3', 'lat', 'lon')

## NOMADS RTOFS

- 1 hour

http://polar.ncep.noaa.gov/mmab/papers/tn284/v211p211.pdf

https://data.nodc.noaa.gov/thredds/catalog/ncep/rtofs/catalog.html

In [7]:
rotfs_url = 'https://data.nodc.noaa.gov/thredds/dodsC/ncep/rtofs/2015/201512/ofs.20151231/surface/ofs_atl.t00z.n000.20151231.grb.grib2'

In [8]:
rotfs_ds = xr.open_dataset(rotfs_url)

In [9]:
rotfs_ds

<xarray.Dataset>
Dimensions:                                    (bounds_dim: 2, hybrid_layer: 1, time: 1, x: 1200, y: 1684)
Coordinates:
    Longitude_of_V_Wind_Component_of_Velocity  (y, x) float32 ...
    Latitude_of_V_Wind_Component_of_Velocity   (y, x) float32 ...
    Latitude_of_U_Wind_Component_of_Velocity   (y, x) float32 ...
    Longitude_of_U_Wind_Component_of_Velocity  (y, x) float32 ...
    Longitude_of_Pressure_Point                (y, x) float32 ...
    Latitude_of_Pressure_Point                 (y, x) float32 ...
  * time                                       (time) datetime64[ns] 2015-12-31
  * hybrid_layer                               (hybrid_layer) float64 1.5
Dimensions without coordinates: bounds_dim, x, y
Data variables:
    V-component_of_current                     (time, hybrid_layer, y, x) float32 ...
    U-component_of_current                     (time, hybrid_layer, y, x) float32 ...
    Barotropic_U_velocity                      (time, y, x) float32 ...
    

In [10]:
rotfs_ds.Longitude_of_U_Wind_Component_of_Velocity

<xarray.DataArray 'Longitude_of_U_Wind_Component_of_Velocity' (y: 1684, x: 1200)>
[2020800 values with dtype=float32]
Coordinates:
    Longitude_of_V_Wind_Component_of_Velocity  (y, x) float32 ...
    Latitude_of_V_Wind_Component_of_Velocity   (y, x) float32 ...
    Latitude_of_U_Wind_Component_of_Velocity   (y, x) float32 ...
    Longitude_of_U_Wind_Component_of_Velocity  (y, x) float32 ...
    Longitude_of_Pressure_Point                (y, x) float32 ...
    Latitude_of_Pressure_Point                 (y, x) float32 ...
Dimensions without coordinates: y, x
Attributes:
    units:                                  degrees_east
    long_name:                              longitude coordinate
    grid_mapping:                           Curvilinear_Orthogonal
    GRIB_param_discipline:                  Meteorological_products
    GRIB_param_category:                    Momentum
    GRIB_param_name:                        Longitude_of_U_Wind_Component_of_...
    GRIB_generating_process_type:

In [11]:
v_current = rotfs_ds.__getitem__('V-component_of_current')
u_current = rotfs_ds.__getitem__('U-component_of_current')
n3d_temperature = rotfs_ds.__getitem__('N3-D_Temperature')

In [12]:
v_current.dims

('time', 'hybrid_layer', 'y', 'x')

In [13]:
n3d_temperature.dims

('time', 'hybrid_layer', 'y', 'x')

### Convert lon, lat to x, y

### Old Examples

In [ ]:
ds = xr.open_dataset('http://nomads.ncep.noaa.gov:9090/dods/rtofs/rtofs_global20180111/rtofs_glo_2ds_forecast_3hrly_diag',
                       chunks={'lat':400, 'lon':400, 'time':10})

In [ ]:
ds

In [ ]:
ds.ssh.isel(time=0).plot()

## Local Files

In [15]:
import netCDF4 as nc

In [17]:
help(nc.Dataset) 

Help on class Dataset in module netCDF4._netCDF4:

class Dataset(builtins.object)
 |  A netCDF `netCDF4.Dataset` is a collection of dimensions, groups, variables and
 |  attributes. Together they describe the meaning of data and relations among
 |  data fields stored in a netCDF file. See `netCDF4.Dataset.__init__` for more
 |  details.
 |  
 |  A list of attribute names corresponding to global netCDF attributes
 |  defined for the `netCDF4.Dataset` can be obtained with the
 |  `netCDF4.Dataset.ncattrs` method.
 |  These attributes can be created by assigning to an attribute of the
 |  `netCDF4.Dataset` instance. A dictionary containing all the netCDF attribute
 |  name/value pairs is provided by the `__dict__` attribute of a
 |  `netCDF4.Dataset` instance.
 |  
 |  The following class variables are read-only and should not be
 |  modified by the user.
 |  
 |  **`dimensions`**: The `dimensions` dictionary maps the names of
 |  dimensions defined for the `netCDF4.Group` or `netCDF4.Dat

In [18]:
nc_file = nc.Dataset('/home/evankielley/Downloads/expt_56.3.nc')

In [28]:
nc_file.['time']

AttributeError: 'netCDF4._netCDF4.Dimension' object has no attribute 'values'

## NAVGEM

- 12 hr
- 0.5 degree

In [ ]:
url = 'http://usgodae.org/dods/GDS/navgem/NAVGEM_0100_010132-000000wnd_ucmp'

In [2]:
url = 'http://usgodae.org/dods/GDS/navgem/NAVGEM_0105_000100-000000wnd_ucmp'

In [3]:
ds = xr.open_dataset(url)

In [4]:
ds

<xarray.Dataset>
Dimensions:     (lat: 361, lon: 720, time: 3604)
Coordinates:
  * time        (time) datetime64[ns] 2013-02-13 2013-02-13T12:00:00 ...
  * lat         (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 -87.5 -87.0 ...
  * lon         (lon) float64 0.0 0.5 1.0 1.5 2.0 2.5 3.0 3.5 4.0 4.5 5.0 ...
Data variables:
    ugrd10m_10  (time, lat, lon) float64 ...
Attributes:
    title:          US058GMET-GR1mdl.0018_0056_00000F0RLTIMESERIES_0105_00010...
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://www.usgodae.org
    history:        Fri Jan 19 18:38:20 UTC 2018 : imported by GrADS Data Ser...

In [5]:
ds['time']

<xarray.DataArray 'time' (time: 3604)>
array(['2013-02-13T00:00:00.000000000', '2013-02-13T12:00:00.000000000',
       '2013-02-14T00:00:00.000000000', ..., '2018-01-18T12:00:00.000000000',
       '2018-01-19T00:00:00.000000000', '2018-01-19T12:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2013-02-13 2013-02-13T12:00:00 2013-02-14 ...
Attributes:
    grads_dim:      t
    grads_mapping:  linear
    grads_size:     3604
    grads_min:      00z13feb2013
    grads_step:     12hr
    long_name:      time
    minimum:        00z13feb2013
    maximum:        12z19jan2018
    resolution:     0.5

In [6]:
ds.ugrd10m_10

<xarray.DataArray 'ugrd10m_10' (time: 3604, lat: 361, lon: 720)>
[936751680 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2013-02-13 2013-02-13T12:00:00 2013-02-14 ...
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 -87.5 -87.0 -86.5 ...
  * lon      (lon) float64 0.0 0.5 1.0 1.5 2.0 2.5 3.0 3.5 4.0 4.5 5.0 5.5 ...
Attributes:
    long_name:  ** 10 m u wind [m/s] 

In [23]:
ds.ugrd10m_10.sel(time='2013-02-13')[0,:10,:10]

<xarray.DataArray 'ugrd10m_10' (lat: 10, lon: 10)>
array([[  3.879152e+03,   3.178646e+30,   1.356316e-19,   4.416006e+21,
          1.601957e-19,   2.107628e-03,   1.358872e-19,   1.928259e+31,
          4.729414e+22,   3.163622e-18],
       [  7.155773e+22,   3.176899e+30,   4.623582e+30,   1.934460e-19,
          1.849871e+20,   1.765821e+28,   4.360454e+21,   1.270943e+31,
          1.141805e+33,   5.393902e+28],
       [  3.974212e+03,   3.025672e+29,   1.440086e+28,   2.745414e+20,
          3.094673e+32,   1.893809e+23,   3.038569e+29,   4.713871e+33,
          4.735676e+22,   7.409798e+28],
       [  1.788248e+28,   6.713558e-07,   2.625702e-06,   2.389918e-06,
          7.271552e+31,   2.113177e-03,            nan,            nan,
                   nan,            nan],
       [           nan,            nan,            nan,            nan,
                   nan,            nan,            nan,            nan,
                   nan,            nan],
       [           nan, 

## ERDAP NAVEGM

In [24]:
url = 'http://coastwatch.pfeg.noaa.gov/erddap/griddap/erdNavgem05D10mWind_LonPM180'

In [25]:
ds = xr.open_dataset(url)

In [26]:
ds

<xarray.Dataset>
Dimensions:                       (height_above_ground: 1, latitude: 361, longitude: 720, time: 6495)
Coordinates:
  * time                          (time) datetime64[ns] 2013-02-15T12:00:00 ...
  * height_above_ground           (height_above_ground) float32 10.0
  * latitude                      (latitude) float32 -90.0 -89.5 -89.0 -88.5 ...
  * longitude                     (longitude) float32 -180.0 -179.5 -179.0 ...
Data variables:
    wnd_ucmp_height_above_ground  (time, height_above_ground, latitude, longitude) float32 ...
    wnd_vcmp_height_above_ground  (time, height_above_ground, latitude, longitude) float32 ...
Attributes:
    _CoordSysBuilder:                     ucar.nc2.dataset.conv.CF1Convention
    cdm_data_type:                        Grid
    Conventions:                          CF-1.6, COARDS, ACDD-1.3
    creator_name:                         FNMOC
    creator_type:                         institution
    creator_url:                          http:

In [27]:
ds['time']

<xarray.DataArray 'time' (time: 6495)>
array(['2013-02-15T12:00:00.000000000', '2013-02-15T18:00:00.000000000',
       '2013-02-16T00:00:00.000000000', ..., '2018-01-18T06:00:00.000000000',
       '2018-01-18T12:00:00.000000000', '2018-01-18T18:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2013-02-15T12:00:00 2013-02-15T18:00:00 ...
Attributes:
    _CoordinateAxisType:  Time
    actual_range:         [  1.36092960e+09   1.51629840e+09]
    axis:                 T
    comment:              Uninitialized analysis / image product / forecast p...
    ioos_category:        Time
    long_name:            Time
    standard_name:        time
    time_origin:          01-JAN-1970 00:00:00

In [29]:
ds.sel(time=slice('2015-06-01','2015-07-01'), longitude=slice(-52,-49), latitude=slice(47,49))

<xarray.Dataset>
Dimensions:                       (height_above_ground: 1, latitude: 4, longitude: 7, time: 120)
Coordinates:
  * time                          (time) datetime64[ns] 2015-06-01 ...
  * height_above_ground           (height_above_ground) float32 10.0
  * latitude                      (latitude) float32 47.5 48.0 48.5 49.0
  * longitude                     (longitude) float32 -52.0 -51.5 -51.0 ...
Data variables:
    wnd_ucmp_height_above_ground  (time, height_above_ground, latitude, longitude) float32 ...
    wnd_vcmp_height_above_ground  (time, height_above_ground, latitude, longitude) float32 ...
Attributes:
    _CoordSysBuilder:                     ucar.nc2.dataset.conv.CF1Convention
    cdm_data_type:                        Grid
    Conventions:                          CF-1.6, COARDS, ACDD-1.3
    creator_name:                         FNMOC
    creator_type:                         institution
    creator_url:                          http://www.usno.navy.mil/FNMOC